In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3
import time

#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()
# Create the first table
all_time_prices_query = '''
CREATE TABLE all_time_prices (
    Date DATETIME,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock_Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

cursor.execute(all_time_prices_query)

conn.commit()

C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Create first table with all stock info from yfinance
tick = pd.read_csv("all_stocks")
all_time_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock_Splits', 'Ticker']
all_time = pd.DataFrame(columns = all_time_cols)
count = 0 
for i in range(len(tick)): 
    start = time.time()    
    try:
        #Creating the df to be added to all_time_prices
        ticker = tick["0"][i]
        info = yf.Ticker(ticker).history(period='max')
        info.reset_index(inplace = True)
        info['Ticker'] = ticker
        info.rename({'Stock Splits':'Stock_Splits'},axis = 1,inplace = True)
        info['Volume'] = info['Volume'].astype(float)
        print(info) 
        
        # Append the first dataframe to the table
        all_time = pd.concat([all_time, info])

        #print(momentum_append)
    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    except AttributeError: 
        print("Attribute error found")
        continue
    stop = time.time()
    duration = stop-start
    print(duration)

#PUSH TO DATABASE
all_time.to_sql('all_time_prices', conn, if_exists='append', index=False)
# Close the connection
conn.close()

                          Date       Open       High        Low      Close  \
0    1983-09-30 00:00:00-04:00   0.000000   0.467834   0.450507   0.450507   
1    1983-10-03 00:00:00-04:00   0.000000   0.453395   0.447619   0.450507   
2    1983-10-04 00:00:00-04:00   0.000000   0.447619   0.444731   0.444731   
3    1983-10-05 00:00:00-04:00   0.000000   0.441843   0.392749   0.392749   
4    1983-10-06 00:00:00-04:00   0.000000   0.401413   0.389862   0.401413   
...                        ...        ...        ...        ...        ...   
9927 2023-02-17 00:00:00-05:00  67.480003  67.480003  65.470001  67.239998   
9928 2023-02-21 00:00:00-05:00  66.589996  67.160004  65.290001  65.669998   
9929 2023-02-22 00:00:00-05:00  65.919998  66.290001  65.150002  65.470001   
9930 2023-02-23 00:00:00-05:00  65.930000  66.150002  64.930000  65.809998   
9931 2023-02-24 00:00:00-05:00  65.110001  65.400002  64.389999  64.870003   

         Volume  Dividends  Stock_Splits Ticker  
0       18000

C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a cop

                         Date       Open       High        Low      Close  \
0   2017-09-21 00:00:00-04:00  14.914390  14.914390  14.884660  14.914390   
1   2017-09-22 00:00:00-04:00  14.924300  14.934209  14.894570  14.904480   
2   2017-09-25 00:00:00-04:00  14.914390  14.963840  14.914390  14.934209   
3   2017-09-26 00:00:00-04:00  14.934209  14.993669  14.934209  14.973848   
4   2017-09-27 00:00:00-04:00  14.983758  14.983758  14.954029  14.963939   
..                        ...        ...        ...        ...        ...   
119 2018-03-15 00:00:00-04:00  14.970000  14.970000  14.970000  14.970000   
120 2018-03-16 00:00:00-04:00  14.970000  14.970000  14.970000  14.970000   
121 2018-03-19 00:00:00-04:00  14.970000  14.970000  14.970000  14.970000   
122 2018-03-20 00:00:00-04:00  14.970000  14.970000  14.970000  14.970000   
123 2017-09-15 00:00:00-04:00        NaN        NaN        NaN        NaN   

      Volume  Dividends  Stock_Splits  Capital Gains Ticker  
0    11205.0 

C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a cop

                          Date      Open      High       Low     Close  \
0    2008-10-10 00:00:00-04:00  4.271626  4.898193  3.420960  4.742694   
1    2008-10-13 00:00:00-04:00  5.030824  5.588788  4.989662  5.511039   
2    2008-10-14 00:00:00-04:00  5.511036  6.448598  5.511036  6.000398   
3    2008-10-15 00:00:00-04:00  5.820859  6.019298  5.764163  5.858657   
4    2008-10-16 00:00:00-04:00  5.929529  7.186306  5.641321  7.186306   
...                        ...       ...       ...       ...       ...   
2462 2007-07-18 00:00:00-04:00       NaN       NaN       NaN       NaN   
2463 2007-10-17 00:00:00-04:00       NaN       NaN       NaN       NaN   
2464 2007-12-27 00:00:00-05:00       NaN       NaN       NaN       NaN   
2465 2008-04-16 00:00:00-04:00       NaN       NaN       NaN       NaN   
2466 2008-07-16 00:00:00-04:00       NaN       NaN       NaN       NaN   

        Volume  Dividends  Stock_Splits  Capital Gains Ticker  
0     187966.0     0.0000           0.0        

C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a cop

                         Date       Open       High        Low      Close  \
0   2009-09-08 00:00:00-04:00  17.081403  17.323502  16.803438  17.323502   
1   2009-09-09 00:00:00-04:00  17.215903  17.565601  17.215903  17.323502   
2   2009-09-10 00:00:00-04:00  17.251767  17.511800  17.099335  17.359367   
3   2009-09-11 00:00:00-04:00  17.395233  17.745828  17.332467  17.664232   
4   2009-09-14 00:00:00-04:00  17.395235  17.700100  17.350402  17.700100   
..                        ...        ...        ...        ...        ...   
336 2008-07-02 00:00:00-04:00        NaN        NaN        NaN        NaN   
337 2008-10-01 00:00:00-04:00        NaN        NaN        NaN        NaN   
338 2008-12-26 00:00:00-05:00        NaN        NaN        NaN        NaN   
339 2009-04-01 00:00:00-04:00        NaN        NaN        NaN        NaN   
340 2009-07-01 00:00:00-04:00        NaN        NaN        NaN        NaN   

       Volume  Dividends  Stock_Splits  Capital Gains Ticker  
0    109459.

C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a cop

                          Date       Open       High        Low      Close  \
0    2009-09-08 00:00:00-04:00  22.381798  22.381798  22.381798  22.381798   
1    2009-09-09 00:00:00-04:00  22.381797  22.381797  22.191313  22.191313   
2    2009-09-10 00:00:00-04:00  22.191313  22.191313  22.191313  22.191313   
3    2009-09-11 00:00:00-04:00  22.191313  22.191313  22.191313  22.191313   
4    2009-09-14 00:00:00-04:00  22.276936  22.277031  22.276936  22.277031   
...                        ...        ...        ...        ...        ...   
3387 2007-04-11 00:00:00-04:00        NaN        NaN        NaN        NaN   
3388 2007-10-10 00:00:00-04:00        NaN        NaN        NaN        NaN   
3389 2008-04-10 00:00:00-04:00        NaN        NaN        NaN        NaN   
3390 2008-10-09 00:00:00-04:00        NaN        NaN        NaN        NaN   
3391 2009-04-09 00:00:00-04:00        NaN        NaN        NaN        NaN   

      Volume  Dividends  Stock_Splits  Capital Gains Ticker  
0

In [ ]:
all_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9932 entries, 0 to 9931
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype                           
---  ------        --------------  -----                           
 0   Date          9932 non-null   datetime64[ns, America/New_York]
 1   Open          9932 non-null   float64                         
 2   High          9932 non-null   float64                         
 3   Low           9932 non-null   float64                         
 4   Close         9932 non-null   float64                         
 5   Volume        9932 non-null   float64                         
 6   Dividends     9932 non-null   float64                         
 7   Stock_Splits  9932 non-null   float64                         
 8   Ticker        9932 non-null   object                          
dtypes: datetime64[ns, America/New_York](1), float64(7), object(1)
memory usage: 775.9+ KB
